In [ ]:
import os
from os.path import join as opj
import nipype.interfaces.utility as util
import nipype.interfaces.ants as ants
import nipype.interfaces.io as io
from nipype.workflows.smri.ants import ANTSTemplateBuildSingleIterationWF
import nipype.pipeline.engine as pe 

# Study specific variables
project_home = '/Users/myelin/Dropbox/Projects/TH_NAR_ASL/proc'
subjects_dir = project_home
raw_dir = project_home + '/raw'
subjects_list = os.listdir(raw_dir)
output_dir = project_home + '/proc'
wkflow_dir = project_home + '/workflows'

In [ ]:
## NODES
# Data handling Nodes
datasource = pe.Node(interface=util.IdentityInterface(fields=['imageList']),
                     run_without_submitting=True,
                     name='InputImages')
datasource.inputs.imageList = input_images
datasource.inputs.sort_filelist = True

datasink = pe.Node(io.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join(requestedPath, "results")

In [ ]:
# Template builder nodes

initAvg = pe.Node(interface=ants.AverageImages(), name='initAvg')
initAvg.inputs.dimension = 3
initAvg.inputs.normalize = True

iteration1 = ANTSTemplateBuildSingleIterationWF('iteration01')

iteration2 = ANTSTemplateBuildSingleIterationWF('iteration02')

In [ ]:
# build the workflow

tbuilder = pe.Workflow(name="ANTSTemplateBuilder")
tbuilder.base_dir = requestedPath

templatebuilder.connect([(datasource,initAvg, [('imageList','images')]),
                         (initAvg, iteration1,[('output_average_image','fixed_image')]),
                         (datasource, iteration1,[('imageList','images')]),
                         (datasource, iteration1, [('passiveImagesDictionariesList','ListOfPassiveImagesDictionaries')]),
                         (iteration1, iteration2, [('template', 'fixed_image')]),
                         (datasource, iteration2, [('imageList', 'images')]),
                         (datasource, iteration2, [('passiveImagesDictionariesList', 'ListOfPassiveImagesDictionaries')]),
                         (iteration2, datasink, [('template', 'PrimaryTemplate')]),
                         (iteration2, datasink, [('passive_deformed_templates', 'PassiveTemplate')]),
                         (initAvg, datasink, [('output_average_image', 'PreRegisterAverage')])
                        ])
templatebuilder.base_dir = opj(wkflow_dir)
templatebuilder.write_graph(graph2use='flat')
templatebuilder.run('MultiProc',plugin_args={'n_procs':2})